In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from trinta_testes_validacao_cruzada import TrintaTestes
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

## Funções para plotagem da matriz de confusão e dos atributos mais importantes

In [ ]:
# Funções para plotagem da matriz de confusão

def view_confusion_matrix(conf_matrix, classes=None,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    accuracy = np.trace(conf_matrix) / np.sum(conf_matrix).astype('float')
    misclass = 1 - accuracy
    if normalize:
        conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    plt.imshow(conf_matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if classes:
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else '.0f'
    thresh = conf_matrix.max() / 2.
    for i, j in itertools.product(range(conf_matrix.shape[0]), range(conf_matrix.shape[1])):
        plt.text(j, i, format(conf_matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()

def plotConfusionMatrix(cnf_matrix, classes_):
    np.set_printoptions(precision=2)

    # para habilitar a configuração do tamanho da imagem a ser plotada 
    matplotlib.style.use('default')
    
    # tamanho da imagem a ser plotada
    plt.figure(figsize=(6,4))
        
    # Plot non-normalized confusion matrix
    if classes_:
        view_confusion_matrix(cnf_matrix, classes=classes_,
                      title='Confusion matrix, without normalization')
    else:
        view_confusion_matrix(cnf_matrix,
                      title='Confusion matrix, without normalization')
    
    # tamanho da imagem a ser plotada
    plt.figure(figsize=(6,4))
    
    # Plot normalized confusion matrix
    if classes_:
        view_confusion_matrix(cnf_matrix, classes=classes_, normalize=True,
                      title='Normalized confusion matrix')
    else:
        view_confusion_matrix(cnf_matrix, normalize=True,
                      title='Normalized confusion matrix')
        

# função para verificação dos atributos mais importante para árvore de decisão e random forest

def importance(importancias):
    
    # juntando os dataframes pertencentes à lista passada como parâmetro em um só dataframe
    importances = pd.concat(importancias, axis=0, join='inner')
    
    # eliminando duplicidade de atributos e unificando por média
    importances = importances.groupby(importances.index).agg('mean')
    
    # ordenando por importância
    importances = importances.sort_values('importance', ascending = False)
    display(importances)

    # imprimindo valores com nome dos atributos, em ordem e através de gráfico em barras
    importances.plot(kind='bar')

## Separação da base em previsores e classe

In [ ]:
base = pd.read_csv('dados_meio_tempo_com_odds.csv').drop(['Unnamed: 0'], axis=1).sample(frac=1).reset_index(drop=True)

previsores = base.iloc[:, 0:35].values
classe = base.iloc[:, 35].values

classes_alvo = ['Visitante', 'Empate', 'Mandante']

# apenas para visualizar em forma de dataframe
classe_df = pd.DataFrame(classe)
previsores_df = pd.DataFrame(previsores)

## Pré-processamento dos dados

In [ ]:
# transformando variáveis categóricas em numéricas usando o LabelEncoder

# para previsores
labelencoder_X = LabelEncoder()
previsores[:, 0] = labelencoder_X.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_X.fit_transform(previsores[:, 1])
previsores[:, 31] = labelencoder_X.fit_transform(previsores[:, 31])


# para classe LabelEncoder não costuma ser necessário
'''labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)'''

pass

In [ ]:
# transformando variáveis categóricas em numéricas e tirando o peso dos valores após a transformação
# através do OneHotEncoder

column_tranformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [0, 1, 31])],remainder='passthrough')
previsores = column_tranformer.fit_transform(previsores)

# apenas para visualizar em forma de dataframe
previsores_onehotencoder_df = pd.DataFrame(previsores)

In [ ]:
# fazendo escalonamento das variáveis (normalização) através do StandardScaler

scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

# apenas para visualizar em forma de dataframe
previsores_StandardScaler_df = pd.DataFrame(previsores)

## Fazendo chamada para algoritmos de Machine Learning

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

classificador = DecisionTreeClassifier()
decision_tree = TrintaTestes(classificador, previsores, classe, columns_)

In [ ]:
print(decision_tree.acuracia().mean())

In [ ]:
importance(decision_tree.importances())

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB

classificador = GaussianNB()
naive_bayes = TrintaTestes(classificador, previsores, classe)

In [ ]:
print(naive_bayes.acuracia().mean())

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# ODDS

from odds import OddsClassifier

classificador = OddsClassifier() 
odds = TrintaTestes(classificador, previsores, classe)

In [ ]:
len(odds.importances())

In [ ]:
print(odds.acuracia().mean())

In [ ]:
plotConfusionMatrix(odds.matrizConfusao(), classes_alvo)

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# KNN

from sklearn.neighbors import KNeighborsClassifier

classificador = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
knn = TrintaTestes(classificador, previsores, classe)

In [ ]:
print(knn.acuracia().mean())

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# RNA

from sklearn.neural_network import MLPClassifier

classificador = MLPClassifier(verbose = True, max_iter = 1000,
                      tol = 0.000010, solver='adam',
                      hidden_layer_sizes=(100), activation = 'relu',
                      batch_size=200, learning_rate_init=0.001)
rna = TrintaTestes(classificador, previsores, classe)

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# SVM

from sklearn.svm import SVC

classificador = SVC(kernel = 'rbf', random_state = 1, C = 2.0)
svm = TrintaTestes(classificador, previsores, classe)

In [ ]:
print(svm.acuracia().mean())

In [ ]:
# chamando método para fazer treinamento e classificação com trinta testes
# e validação cruzada usando StratifiedKFold
# Random Forest

from sklearn.ensemble import RandomForestClassifier

columns_ = base.iloc[:, 0:35].columns
classificador = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
random_forest = TrintaTestes(classificador, previsores, classe, columns_)

In [ ]:
importance(random_forest.importances())

In [ ]:
print(random_forest.acuracia().mean())

## Lista com os resultados de cada algoritmo

In [ ]:
# Lista com os resultados dos algoritmos para o boxplot - trazida por chamada anteriormente
# para não ter que executar todo o código sempre

random_forest_list = [0.6440528058556989, 0.6353738236319275, 0.640083652840711, 0.6407764029278494, 0.6433426280934122, 0.6420834785639595, 0.6552631578947368, 0.6334349947716975, 0.6346985012199372, 0.638027187173231, 0.6400400836528407, 0.6393386197281282, 0.6426324503311258, 0.6334872777971419, 0.6452553154409202, 0.6354217497385848, 0.6261327988846288, 0.6346897873823633, 0.6387460787730916, 0.6261937957476473, 0.6346287905193447, 0.6399616591146742, 0.635439177413733, 0.6406326246078773, 0.64394823980481, 0.6354435343325201, 0.6367506099686302, 0.642022481700941, 0.6394170442662949, 0.6414212269083304]
odds_list = [0.554247995817358, 0.55411293133496, 0.5542305681422098, 0.554139072847682, 0.5541303590101081, 0.5541652143604043, 0.5541085744161729, 0.5541521436040433, 0.554126002091321, 0.5541782851167654, 0.5541477866852562, 0.554139072847682, 0.554121645172534, 0.5542262112234229, 0.5541565005228303, 0.5541434297664691, 0.5541521436040433, 0.5541434297664691, 0.5542523527361449, 0.5542044266294877, 0.5540867898222377, 0.5540955036598118, 0.554130359010108, 0.5541565005228303, 0.5541565005228303, 0.5541521436040433, 0.5542000697107006, 0.5541957127919136, 0.5541957127919136, 0.5541957127919136]


### Chamando os resultados e armazenando em lista

In [ ]:
# Trazendo por chamada a lista com os resultados dos algoritmos para o boxplot

random_forest_list = random_forest.acuracia()
odds_list = odds.acuracia()

## Juntando todas as listas em um único dataframe para exibir com o boxplot

In [ ]:
# juntando todas as listas em um só dataframe para plotar com o boxplot

random_forest_df = pd.DataFrame(random_forest_list, columns=['Random Forest'])
#svm_df = pd.DataFrame(svm.acuracia()).rename(columns={0:'SVM'})
#knn_df = pd.DataFrame(knn.acuracia()).rename(columns={0:'KNN'})
#naive_bayes_df = pd.DataFrame(naive_bayes.acuracia()).rename(columns={0:'Naive Bayes'})
#decision_tree_df = pd.DataFrame(decision_tree.acuracia()).rename(columns={0:'Árvore de Decisão'})
odds_df = pd.DataFrame(odds_list, columns=['ODDS'])
# juntando todos os dataframes acima em um só 
resultados_df = pd.concat([random_forest_df,odds_df], axis=1, join='inner')

display(resultados_df)

## Boxplot com os resultados

In [ ]:
fig = plt.figure(figsize=(8,5))
resultados_df.boxplot(column=['Random Forest','ODDS'])
plt.title('Algoritmos de Machine Learning')
plt.ylabel('Acurácia', fontsize=12, color='black')
plt.show()